In [56]:
import pandas as pd

In [57]:
df=pd.read_csv("spotify_millsongdata.csv")

In [58]:
df.head(10)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
5,ABBA,Burning My Bridges,/a/abba/burning+my+bridges_20003011.html,"Well, you hoot and you holler and you make me ..."
6,ABBA,Cassandra,/a/abba/cassandra_20002811.html,Down in the street they're all singing and sho...
7,ABBA,Chiquitita,/a/abba/chiquitita_20002978.html,"Chiquitita, tell me what's wrong \r\nYou're e..."
8,ABBA,Crazy World,/a/abba/crazy+world_20003013.html,I was out with the morning sun \r\nCouldn't s...
9,ABBA,Crying Over You,/a/abba/crying+over+you_20177611.html,I'm waitin' for you baby \r\nI'm sitting all ...


In [59]:
 df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [60]:
df.shape

(57650, 4)

In [61]:
df = df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [62]:
df['text'][0]

"Looking at you now, you seem just a shadow  \r\nYou had the energy to recharge our batteries  \r\nI suppose your success became your security  \r\nDarkness is not what you need  \r\n  \r\nYou were the one who refused to give up  \r\nYou were the one who lifted us, lifted us off our knees  \r\nNow it's our turn to give you some help  \r\nFresh air that's just what you need  \r\n  \r\nNot the stale bread from the failing bitter table  \r\nIt's so hard  \r\nWhere's the justice we cry  \r\nIt's there and will follow us 'til the day we die, 'til  \r\nThe day that we die  \r\n  \r\nLooking at you know, you seem just a shadow  \r\nYou had the energy and you recharged our batteries  \r\nNow it's our turn to give you some help  \r\nFresh air that's just what you need  \r\n  \r\nNow it's our turn to give you some help  \r\nFresh air that's just what you need  \r\nThat's just what you need\r\n\r\n"

In [63]:
df = df.sample(5000)

In [78]:
df.tail(5)

,artist,song,text
4938,George Michael,Love's In Need Of Love Today,good morn or evening friends \r here's your f...
4694,Christina Aguilera,What I'm Looking For,you broke my heart \r that's for sure \r but...
4392,Tori Amos,Carry,"love, hold my hand \r help me see with the da..."
2590,David Bowie,Buddha Of Suburbia,living in lies by the railway line \r pushing...
2032,Steely Dan,A Horse In Town,left new hyde park \r to hold my own in new y...


Text CLeaning /Text Preprocessing

In [65]:
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ', regex=True)

In [66]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')

In [69]:
matrix = tfidvector.fit_transform(df['text'])

In [70]:
similarity = cosine_similarity(matrix)

In [71]:
similarity[0]

array([1.        , 0.07450748, 0.01376049, ..., 0.        , 0.00546336,
       0.00678036], shape=(5000,))

RECOMMENDATION FUNCTION

In [80]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [81]:
recommendation("What I'm Looking For")

['Party',
 'Neverland',
 'Change You Or Change Me',
 'You Were Right Girl',
 'Only Girl',
 'Died And Came Back',
 'Get Right',
 'Family Values',
 'Back To You',
 "Don't Let Go",
 'Please, Please, Please, Let Me Get What I Want',
 'That Girl',
 "I'll Be There For You",
 'Burn',
 'What They Want',
 "That'z Who I Am",
 "Ninety-Nine And A Half (Won't Do)",
 'My Oedipus Complex',
 'Why Wait',
 'Earthquake']

In [82]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))